In [4]:
## 环境
import sys,  os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ["NUMBA_NUM_THREADS"] = "12"
os.environ["NUMBA_THREADING_LAYER"] = "omp"
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
sys.path.append("../")

## 基础
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

## 必要库
from threeML import *
try:
    from hawc_hal import HAL, HealpixConeROI, HealpixMapROI
except:
    from WCDA_hal import HAL, HealpixConeROI, HealpixMapROI
import mylib as my
from importlib import reload

##可选
# from threeML import silence_progress_bars, activate_progress_bars, toggle_progress_bars
# from threeML.utils.progress_bar import trange
# from tqdm import tqdm
# from astropy.io import fits
# from astropy.wcs import WCS
# from astropy.coordinates import SkyCoord
# from astropy import units as u
# from astropy.visualization import astropy_mpl_style, imshAow_norm
# from astropy.coordinates import Angle
import healpy as hp
# import traceback

## 警告
# import warnings
# np.seterr(all="ignore")
np.seterr(divide="ignore")
# warnings.simplefilter("ignore")
# silence_warnings()
# silence_logs()
# quiet_mode()
# loud_mode()
# debug_mode()

Load Mymap:  11%|█         | 4/38 [00:00<00:00, 38.68it/s]     
WARNING VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.


WARNING SyntaxWarning: "is not" with a literal. Did you mean "!="?

Load sub from Mystat: : 40it [00:00, 75.62it/s]                            

Yourlib init successfully!!!


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [ ]:
#####   Data Initialize

# maptree = "../../data/gcd_new.root"

# maptree =  "../../data/20210305_20230731_ihep_goodlist.root"
# maptree =  "../../data/20210305_20230731_ihep_no_goodlist.root"
# maptree =  "../../data/20210305-20230731_trans_fromhsc.root"
# maptree =  "../../data/WCDA_20240131_out.root"

# response = "../../data/DR_ihep_MK2.root"
# response = "../../data/DR_ihep_MK2_newpsf.root"

# response = "/data/home/cwy/Science/3MLWCDA/data/DR_ihep_20240131_Crabpsf_mc.root"
# response = "../../data/DR_ihep_MK2_newoldDRpsf.root"
# response = "../../data/WCDA_DR_psf.root"
# response = "../../data/DR_ihep_mk_luoyu.root"
# response = "../../data/DR_ihep_mk_MC1.root"
# response = "../../data/DR_ihep_mk_MC2.root"
# response = "../../data/DR_ihep_MK2_luoyu.root"

maptree = f"{my.libdir}/../../data/20240731_hsc_out.root"
response= f"{my.libdir}/../../data/DR_ihep_20240131_hscpsf_mc.root"

region_name="S147"
if not os.path.exists(f'{my.libdir}/../res/{region_name}/'):
    os.system(f'mkdir {my.libdir}/../res/{region_name}/')
ra_S147, dec_S147 =   83, 26 #84.60, 28.29
ra1,dec1=ra_S147, dec_S147

data_radius = 7.0
model_radius = 8.0

# roimap = my.getmaskedroi(ra1, dec1, data_radius, [(87.28, 23.90, 2), (83.63, 22.02, 3)])
roimap = my.getmaskedroi(ra1, dec1, data_radius, [(85.78, 23.40, 3), (83.63, 22.02, 3)]) #
roi = HealpixMapROI(ra=ra1,dec=dec1, data_radius=data_radius, model_radius=model_radius, roimap=roimap)

# roi=HealpixConeROI(ra=ra1,dec=dec1,data_radius=data_radius,model_radius=model_radius) 

WCDA = HAL("WCDA", maptree, response, roi, flat_sky_pixels_size=0.17)

#####   Data Situation
%matplotlib inline
WCDA.set_active_measurements(0, 6)
WCDA.display()

fig = WCDA.display_stacked_image(smoothing_kernel_sigma=0.25)
fig.show()
fig.savefig(f"{my.libdir}/../res/{region_name}/counts_all.png",dpi=300)

15:29:15 INFO      Using transits contained in maptree                                                    ]8;id=357518;file:///home/shenhuafei/miniconda3/envs/3MLhal/lib/python3.9/site-packages/hawc_hal/HAL.py\HAL.py]8;;\:]8;id=116175;file:///home/shenhuafei/miniconda3/envs/3MLhal/lib/python3.9/site-packages/hawc_hal/HAL.py#78\78]8;;\

OSError: MapTree ../../data/20240731_hsc_out.root does not exist or is not readable

mkdir: cannot create directory ‘../res/S147/’: No such file or directory


Load sub from Mystat: : 40it [00:20, 75.62it/s]

In [ ]:
map2, skymapHeader = hp.read_map("/data/home/cwy/Science/3MLWCDA/Standard/res/S147/Halpha_fwhm06_1024.fits",h=True)
map2 = my.change_coord(map2, ["G","C"])
my.heal2fits(map2, "/data/home/cwy/Science/3MLWCDA/Standard/res/S147/S147_HI_temp.fits", ra_min = 82, ra_max = 88, xsize=0.01, dec_min=26, dec_max=30, ysize=0.01, nside=1024, ifplot=0, check=0, alpha=1, ifnorm=True)
figfits, wcs, fitsdata = my.drawfits(fits_file_path="/data/home/cwy/Science/3MLWCDA/Standard/res/S147/S147_HI_temp.fits")

In [ ]:
%matplotlib inline
bestmodel, result = my.Search(ra1, dec1, data_radius, model_radius,  region_name+"_new_freeDGE0-5", WCDA, roi, 0, 5,freeDGE=1, fromcatalog=1, fixcatall=1)

In [ ]:
Modelname="cat+1ext+1pt_freeDGE_0-6"
if not os.path.exists(f'{my.libdir}/../res/{region_name}/{Modelname}/'):
    os.system(f'mkdir {my.libdir}/../res/{region_name}/{Modelname}/')

lm = my.getcatModel(ra1, dec1, data_radius, model_radius, roi=roi)
with open(f'{my.libdir}/../res/{region_name}/{Modelname}/Model_init.txt', 'w') as f:
    for key, value in lm.parameters.items():
        f.write(f'{key}: {value}\n')

# S147 = my.set_diffusemodel("S147", '/data/home/cwy/Science/3MLWCDA/Standard/res/S147/S147_HI_temp.fits', K=7.3776826e-14, Kb=(1e-17, 1e-10))
S147 = my.setsorce("S147",84.5,28.5, sigma=0.2, sb=(0,5), #sf=True, #raf=True,decf=True,
                indexb=(-4,-1),kb=(1e-16, 1e-12), #indexf=True,
                fitrange=1)

PWN = my.setsorce("PWN",84.5,28.5, #sf=True, #raf=True,decf=True,
                indexb=(-4,-1),kb=(1e-16, 1e-12), #indexf=True,
                fitrange=1)

Diffuse = my.set_diffusebkg(
                ra1, dec1, data_radius, data_radius, name=region_name+"WCDA",
                Kf=False, indexf=False
                )
# lm = Model(S147, PWN, Diffuse)
lm.add_source(S147)
lm.add_source(PWN); lm.add_source(Diffuse)

lm.save(f"{my.libdir}/../res/{region_name}/{Modelname}/Model_init.yml", overwrite=True)

lm.display(complete=True)
        

In [ ]:
my.getresaccuracy(WCDA, lm, plot=True, savepath=f"{my.libdir}/../res/{region_name}/{Modelname}/")

In [ ]:
reload(my)

In [ ]:
my.get_residual_significance_mapfast(WCDA, lm=None, combine='weighted', plot=True, savepath=f"{my.libdir}/../res/{region_name}/{Modelname}/")

In [ ]:
%matplotlib inline
result = my.fit(region_name, Modelname, WCDA, lm, 0, 6,mini="ROOT")
sources = my.get_sources(lm,result)
result[1][0]
# result[0].get_contours(J0248.Gaussian_on_sphere.lon0,39.88,44.88,30)

In [ ]:
TSlist = ["S147", "PWN", "Diffuse"] #
TS, TSresults = my.getTSall(TSlist, region_name, Modelname, result, WCDA)
TSresults

In [ ]:
reload(my)

In [ ]:
%matplotlib inline
sources = my.get_sources(lm,result) #
sources.pop("Diffuse")
# sources.pop("S147")
# sources={}

map2, skymapHeader = hp.read_map(f"{my.libdir}/../../data/fullsky_WCDA_20240731_0-6_2.6.fits.gz",h=True)
map2 = my.maskroi(map2, roi)

fig = my.drawmap(region_name, Modelname, sources, map2, ra1, dec1, rad=2*data_radius, contours=[3],save=1, 
                cat={"PSR": [0, "*"],"SNR": [0, "o"],"3FHL": [0, "D"],"4FGL": [0, "d"],"size": 20,"markercolor": "grey","labelcolor": "black","angle": 60,"catext": 1  }, #
                # cat={ "LHAASO": [0, "P"],"TeVCat": [0, "s"],"PSR": [0, "*"],"SNR": [0, "o"],"3FHL": [0, "D"], "4FGL": [0, "d"], "YMC": [0, "^"], "GYMC":[0, "v"], "WR":[0, "X"], "size": 20, "color": "grey", "angle": 60, "catext": 1}, #2*data_radius
                # color="Milagro"
                  color="Fermi",
                  # Drawdiff=1, legend=0,
                  # ifdrawfits=1, cutl=0, alphaf=1, fitsfile="/data/home/cwy/Science/3MLWCDA/Standard/res/S147/S147_HI_temp.fits" #, vmin=-15, vmax=30, #
                  )
# ax = plt.gca()
# ax.set_aspect('equal', adjustable='box')

In [ ]:
my.write_resmap(region_name, Modelname, WCDA, roi, maptree, ra1, dec1, "S147_res_all",[0],[0,0])

In [ ]:
Flux_WCDA0, jls0  = my.getdatapoint(WCDA, lm, maptree, response, roi, "S147", ifpowerlawM=1)
Flux_WCDA1, jls1  = my.getdatapoint(WCDA, lm, maptree, response, roi, "PWN", ifpowerlawM=1)

In [ ]:
%matplotlib inline
import matplotlib as mpl

x_Max=50.
x_Min=0.5
y_Min=0.1e-13
y_Max=8e-12
fig,ax = plt.subplots()
plot_spectra(
    result[0].results,
    sources_to_use=["S147","PWN"], # 
    include_extended=True,
    ene_min=x_Min,
    ene_max=x_Max,
    num_ene=30,
    energy_unit="TeV",
    flux_unit="TeV/(s cm2)",
    subplot=ax,
    )

my.Draw_sepctrum_points(region_name, Modelname, Flux_WCDA0,"S147","tab:blue")
my.Draw_sepctrum_points(region_name, Modelname, Flux_WCDA1,"PWN","tab:red")

# LHAASO_sensitivity = np.genfromtxt("./LHAASO_sensitivity.csv",delimiter=',')
# plt.plot(LHAASO_sensitivity[:,0],LHAASO_sensitivity[:,1]*1e-11,label="LHAASO 1yr", c="black")

ax.set_xlim(x_Min,x_Max)
ax.set_ylim(y_Min,y_Max)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r"$E^2\,dN/dE$ [TeV cm$^{-2}$ s$^{-1}$]")
ax.set_xlabel("Energy [TeV]")
plt.legend()
plt.savefig(f'{my.libdir}/../res/{region_name}/{Modelname}/Spectrum.png', dpi=300)
plt.savefig(f'{my.libdir}/../res/{region_name}/{Modelname}/Spectrum.pdf')